### Imports

In [44]:
import torch
import torch.nn as nn
import networks
import datasetLoader
import evaluation
import importlib
import matplotlib.pyplot as plt

importlib.reload(datasetLoader)
importlib.reload(networks)
importlib.reload(evaluation)


<module 'evaluation' from 'c:\\Users\\trist\\Git_repos\\BT-ML-PGESAT\\code\\PGExplainer\\evaluation.py'>

### Testing PyG

In [2]:
from torch_geometric.data import Data

edge_index = torch.tensor([[0, 1, 1, 2],
                           [1, 0, 2, 1]], dtype=torch.long)
x = torch.tensor([[-1], [0], [1]], dtype=torch.float)

data = Data(x=x, edge_index=edge_index)
print(data)

Data(x=[3, 1], edge_index=[2, 4])


### Parameters

In [48]:
batch_size = 1




learning_rate_gnn = 0.001        #0.001 on ADAM
epochs_graphgnn = 5000
epochs_nodegnn = 1000


learning_rate_mlp = 0.003        #0.003 on ADAM
coefficientSizeReg = 0.05
entropyReg = 1
epochs_mlp = 30

temperature =  0        #???


## Training Loop GNN

In [ ]:
train_loader, val_loader, test_loader = datasetLoader.loadDataset('BA2Motif', batch_size)               # TODO: adjs matrix is not compatible with GraphConv. Needs to be converted to edge_index (see RE_PGE datasets/utils)

temp = next(iter(train_loader))
gnn = networks.GraphGNN(features = temp.x.shape[1], labels=2)       # temp.y.shape[0] is wrong!!! TODO: how do I get #labels from loader??


gnn_optimizer = torch.optim.Adam(params = gnn.parameters(), lr = learning_rate_gnn)         # TODO: understand params

print(f"Training on {len(train_loader) * batch_size} graphs with batch size {batch_size}")
print(len(val_loader))

# TODO: adapt this to work on batch_sizes > 1 !!!

loss = nn.CrossEntropyLoss()           # cross entropy loss?!

for epoch in range(0, epochs_nodegnn) :
    print(f'\n------------------ EPOCH {epoch + 1} -------------------')

    gnn.train()

    train_acc_sum = 0
    
    for batch_index, data in enumerate(train_loader):
        
        train_loss = 0.0
        gnn_optimizer.zero_grad()       # Reset parameters

        # real label
        label = data.y

        # get model embeddings (node representations)?
        # predicted label
        out = gnn.forward(data.x, data.edge_index, data.batch)

        # calc cross entropy(???)loss between real label and predicted label
        # needs to be calculated across batch
        #print(out.argmax(dim=1))
        currLoss = loss(out, label)

        # loss backward
        currLoss.backward()
        
        torch.nn.utils.clip_grad_norm_(gnn.parameters(), max_norm=2)    # clip gradient above 2(for ba2motfis according to reimplementation) to stop "overlearning"?

        # optimizer step
        gnn_optimizer.step()

        preds = out.argmax(dim=1)
        train_acc_sum += torch.sum(preds == data.y)         # TODO: works with batches?
        
        train_loss += currLoss.item()

    gnn.eval()

    final_train_acc = train_acc_sum/(len(train_loader)*batch_size)              # TODO: see evaluation.py, not quite correct

    # avg loss
    print(f"average training loss: {train_loss/len(train_loader)}, training acc: {final_train_acc}")

    test, val_acc, valLoss, test_acc, testLoss = evaluation.evaluateGNN(gnn, val_loader, test_loader)
    print(test)
    print(f"validation loss: {valLoss}, validation acc: {val_acc}, test loss: {testLoss}, test acc: {test_acc}")
    # Evaluation per epoch
    # TODO: implement evaluation methods that wprk on batch_sizes > 1
    """gnn.eval()

    num_batches = 0.0

    gnn_losses = 0.0

    test_acc_sum = 0

    for batch_index, data in enumerate(test_loader):
        batch_size_ratio = len(data)/batch_size
        num_batches += batch_size_ratio

        with torch.no_grad():
            out = gnn.forward(data.x, data.edge_index, data.batch)
            gnnLoss = loss(out, data.y)

            preds = out.argmax(dim=1)

            test_acc_sum += torch.sum(preds == data.y)
            

        gnn_losses += batch_size_ratio * gnnLoss

    final_test_acc = test_acc_sum/len(test_loader)
    print(f"testing accuracay: {final_test_acc}")
    print(f"testing loss average: {gnn_losses/num_batches}")"""


# move training loop Explainer
#mlp_optimizer = torch.optim.Adam(lr = learning_rate_mlp)


"""for i in enumerate(adjs):
    #out = gnn.forward(feas[i], adjs[i].nonzero().t().contiguous())

for epoch in epochs_graphgnn:
    for graph in adjs:
        # calculate latent variables? MLP?
        for k in # k in monte carlo sampling?!
            # sammple graph
            # pred label on sampled graph

    # compute loss
    # update params with backprop"""

    

Training on 800 graphs with batch size 1
100

------------------ EPOCH 1 -------------------
average training loss: 0.0008873338997364044, training acc: 0.5287500023841858
tensor(65.2019)
validation loss: 0.6520191431045532, validation acc: 0.5199999809265137, test loss: 0.6530731320381165, test acc: 0.49000000953674316

------------------ EPOCH 2 -------------------
average training loss: 0.0004222356528043747, training acc: 0.6675000190734863
tensor(30.2551)
validation loss: 0.30255112051963806, validation acc: 0.9399999976158142, test loss: 0.3106735944747925, test acc: 0.9300000071525574

------------------ EPOCH 3 -------------------
average training loss: 1.6606738790869713e-06, training acc: 0.9325000047683716
tensor(2.0028)
validation loss: 0.02002844400703907, validation acc: 1.0, test loss: 0.03854673355817795, test acc: 0.9900000095367432

------------------ EPOCH 4 -------------------
average training loss: 3.2365109655074775e-06, training acc: 0.9950000047683716
tensor(0.5

KeyboardInterrupt: 

##### Saving Models

In [ ]:
torch.save(gnn.state_dict(), f"models/BA2Motif")

##### Loading Models

In [ ]:
#model = TheModelClass(*args, **kwargs)
#model.load_state_dict(torch.load(PATH, weights_only=True))
#model.eval()

In [ ]:
print(len(test_loader))

print(next(iter(train_loader)))         # edge_index = "map" for edges, x = features, y = labels
#print(next(iter(train_loader)).x)

print("test_loader contains 100 graphs with following labels:")
for i, curr in enumerate(test_loader):
    print(curr.y)
    print(curr.x.shape[1])
    print(curr.batch)

    out = gnn.forward(curr.x, curr.edge_index, curr.batch)
    print(out.)

100
DataBatch(x=[25, 10], edge_index=[2, 52], y=[1], batch=[25], ptr=[2])
test_loader contains 100 graphs with following labels:
tensor([0])
10
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0])
tensor([[0.2440]], grad_fn=<AddmmBackward0>)
tensor([0])
10
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0])
tensor([[0.2392]], grad_fn=<AddmmBackward0>)
tensor([1])
10
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0])
tensor([[0.2350]], grad_fn=<AddmmBackward0>)
tensor([0])
10
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0])
tensor([[0.2379]], grad_fn=<AddmmBackward0>)
tensor([1])
10
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0])
tensor([[0.2447]], grad_fn=<AddmmBackward0>)
tensor([0])
10
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0])
tens

## Testing

In [34]:
from torch_geometric.datasets import ExplainerDataset
from torch_geometric.datasets.graph_generator import BAGraph
from torch_geometric.datasets.motif_generator import HouseMotif
from torch_geometric.datasets.motif_generator import CycleMotif
import torch_geometric.transforms as T
from torch_geometric.loader import DataLoader

dataset1 = ExplainerDataset(
            graph_generator=BAGraph(20, 1),
            motif_generator=HouseMotif(),
            num_motifs=1,
            num_graphs=400,
            transform=T.Constant()      # appends value 1 node feature for every node
        )

dataset2 = ExplainerDataset(
            graph_generator=BAGraph(20, 1),
            motif_generator=CycleMotif(5),
            num_motifs=1,
            num_graphs=400,
            transform=T.Constant()
        )

dataset = torch.utils.data.ConcatDataset([dataset1, dataset2])

print(dataset)
dataset[0].y = torch.tensor([0])

print(dataset[0].y)

train_loader = DataLoader(dataset1, batch_size = 1, shuffle = True)

print(next(iter(train_loader)).y)

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 2, 2,
        3])
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 2, 2,
        3])


In [40]:
generator1 = torch.Generator().manual_seed(42)
generator2 = torch.Generator().manual_seed(42)
set1, set2 = torch.utils.data.random_split(range(10), [3, 7])
set3, set4, set5 = torch.utils.data.random_split(range(30), [0.3, 0.3, 0.4])

print("Set 1:")
print(set1[0])
print(set1[1])
print(set1[2])
print("Set 2:")
print(set2[0])
print(set2[1])
print(set2[2])
print(set2[3])
print(set2[4])
print(set2[5])
print(set2[6])

Set 1:
9
3
5
Set 2:
2
6
4
8
7
0
1
